<a href="https://colab.research.google.com/github/soniaeya/ENG_SPA_Transformer_Translator/blob/main/ENG_SPA_Transformer_Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assigment 2: Transformers for Translation 🙊


Have you ever wondered how applications like Google Translate or language translation features in social media platforms work? Behind these impressive technologies are sophisticated machine learning models that can understand and translate text between different languages. One of the most powerful and groundbreaking models used for this purpose is the Transformer model.

In this assignment, you will step into the shoes of an AI researcher and engineer to create your own Transformer model for translating text from English to Spanish. This journey will not only enhance your understanding of machine learning and deep learning but also give you hands-on experience with state-of-the-art techniques in natural language processing.

Let's start by downloading important libraries

### Intro


In [1]:
!pip install datasets
!pip install evaluate
!pip install transformers
!pip install bert_score
!pip install rouge_score

For this assignment we are using the Opus Book dataset (read more about it [here](https://huggingface.co/datasets/Helsinki-NLP/opus_books) ). This dataset easily found in huggingface fits perfectly for our machine translation task.

In [2]:
from datasets import load_dataset

books = load_dataset("opus_books", "en-es")
books = books["train"].train_test_split(test_size=0.2)
train_dataset = books["train"]['translation']
test_size = int(len(books["test"])/2)
val_dataset = books["test"][:test_size]['translation']
test_dataset = books["test"][test_size:]['translation']

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Just to have an idea let's have a quick peak on how our dataset looks like

In [3]:
train_dataset[0]

{'en': 'She had small round, hanging gold earrings, and a general air of being fairly well-to-do in a vulgar, comfortable, easy-going way."',
 'es': 'Llevaba pendientes de oro, pequeños y redondos, y en general tenía aspecto de persona bastante bien acomodada, con un estilo de vida vulgar, cómodo y sin preocupaciones.'}

Since we don't want to take 8 hours training let's trim our dataset a bit (although this might lead to underperformance, feel free to use the complete dataset if you have the computing power).

SUGESTION: start with a small dataset to debug your code and increase it gradually (same applies with number of epochs, batch size, test set size)

In [4]:
train_dataset = train_dataset[:30000]

### Preprocessing


Preprocessing is an important part of NLP. This allows us to clean and standarize our data.

In [5]:
import string
import re
def preprocess_data(text):
  """ Method to clean text from noise and standarize text.
  Arguments
  ---------
  text : List of String
     Text to clean
  Returns
  -------
  text : String
      Cleaned and joined text
  """

  text = text.lower()

  text= re.sub(r'[^\w\sÁÉÍÓÚáéíóúüÜñÑ]', '', text) #remove any punctuation or special characters without taking off accent letters like ó,í,etc..
  text = re.sub(r"\s+", " ", text).strip() #to remove extra space (I THINK CAN REMOVE)

  return text


In [6]:
assert "para ti es una cuestión de tozudez dijo ana de repente al encontrar una palabra que definiera justamente los pensamientos y el sentir de vronsky un calificativo para aquella expresión de su rostro que tanto la irritaba"==preprocess_data("–Para ti es una cuestión de tozudez –dijo Ana de repente, al encontrar una palabra que definiera justamente los pensamientos y el sentir de Vronsky, un calificativo para aquella expresión de su rostro que tanto la irritaba–."), "Check errors in preprocessing"
print("Good Job!")

Good Job!


For an easier training structure, it is useful to format our training and validation sets. The following function should help with this.

In [7]:
def create_dataset(dataset,source_lang,target_lang):
  """ Method to create a dataset from a list of text.
  Arguments
  ---------
  text : List of String
     Text from dataset
  source_lang : String
     Source language
  target_lang : String
     Target language
  Returns
  -------
  new_dataset : Tuple of String
      Source and target text in format (source, target)
  """
  new_dataset=[]

#Iterate through dataset extract source and target dataset and preprocess them creating a new clean dataset with the correct format
  for i in dataset:
    # Fix: Changed tuple[i[source_lang], i[target_lang]] to (i[source_lang], i[target_lang]) to correctly create a tuple.
    new_dataset.append((i[source_lang], i[target_lang]))


  return new_dataset

training_set=create_dataset(train_dataset,'en','es')
validation_set=create_dataset(val_dataset,'en','es')
test_set=create_dataset(test_dataset,'en','es')

### Model Creation


Now that our data is ready, we can get started. Let's start by creating our Sequence to Sequence Transformer model.

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class TransformerModel(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward,dropout):
        super(TransformerModel, self).__init__()
        self.src_embedding = nn.Embedding(src_vocab_size, d_model) # Embedding layer for source language | Lookup table of shape [src_vocab_size, d_model]
        self.tgt_embedding = nn.Embedding(tgt_vocab_size,d_model) # Embedding layer for target language| Lookup table of shape [tgt_vocab_size, d_model]
        self.transformer = nn.Transformer(d_model, nhead, num_encoder_layers, num_decoder_layers,dim_feedforward,dropout, batch_first=True)# Transformer model with it's attributes (see pytorch documentation)
        self.fc = nn.Linear(d_model, tgt_vocab_size) # Last linear layer
        self.d_model = d_model
    def positional_encoding(self, d_model, maxlen = 5000):
        """Method to create a positional encoding buffer.
        Arguments
        ---------
        d_model: int
            Embedding size
        maxlen: int
            Maximum sequence length
        Returns
        -------
        PE: Tensor
            Positional encoding buffer
        """
        pos = torch.arange(0, maxlen).unsqueeze(1)
        pos = pos.float()
        denominator = 10000 ** (torch.arange(0, d_model, 2) / d_model) #torch.arange(start, end, step)->create a 1D   tensor with evenly spaced value

        PE = torch.zeros((maxlen, d_model))
        PE[:, 0::2] = torch.sin(pos/denominator)  # Calculate sin for even positions
        PE[:, 1::2] = torch.cos(pos/denominator) # Calculate cosine odd positions

        PE = PE.unsqueeze(0)  # Add batch dimension
        return PE


    def forward(self, src, tgt, src_mask=None, tgt_mask=None, src_key_padding_mask=None, tgt_key_padding_mask=None):
        #Emb weight matrix (self.src_embedding) -> [src_vocab_size, d_model]
        #Input token indices (src)              -> [batch_size, seq_len]
        #Output embedding (src_emb)             -> [batch_size, seq_len, d_model]
        """Method to forward a batch of data through the model."""

        scale = math.sqrt(self.d_model)

        #pass source and target throught embedding layer
        src_emb = self.src_embedding(src) * scale #size -> (batch, seq_len, d_model)
        tgt_emb = self.tgt_embedding(tgt) * scale #size -> (batch, seq_len, d_model)

        #create positional encoding
        pe = self.positional_encoding(self.d_model).to(src.device) # (batch, seq, d_model)



        #slices the positional encoding so it matches the sequence length of x.
        #Bcz PEs must match the length of the input sequence
        src_seq_len = src_emb.size(1)   # returns seq length
        tgt_seq_len = tgt_emb.size(1)

        src_pe = pe[:, :src_seq_len, :]
        tgt_pe = pe[:, :tgt_seq_len, :]

        #add positional encoding to src and tgt
        src_emb = src_emb + src_pe # (batch, seq, d_model)
        tgt_emb = tgt_emb + tgt_pe # (batch, seq, d_model)

        #pass src, tgt and all masks throught transformer
        # Pass through Transformer
        out = self.transformer(
            src=src_emb,
            tgt=tgt_emb,
            src_mask=src_mask,
            tgt_mask=tgt_mask,
            src_key_padding_mask=src_key_padding_mask,
            tgt_key_padding_mask=tgt_key_padding_mask,
        )  # [batch, tgt_len, d_model] (because batch_first=True)

        #pass output throught linear layer
        output = self.fc(out)
        return output

    def encode(self, src, src_mask):
        """Method to encode a batch of data through the transformer model."""
        # encode() takes in a sentence

        #pass src throught embedding layer
        scale = math.sqrt(self.d_model)
        src_emb = self.src_embedding(src) * scale #size -> (batch, seq_len, d_model)

        src_seq_len = src_emb.size(1)   # returns seq length
        #create positional encoding


        pe = self.positional_encoding(self.d_model, maxlen=src_seq_len).to(src.device) # (batch, seq, d_model)
        src_pe = pe[:, :src_seq_len, :]
        #add src_emb and src_pe
        src_emb = src_emb + src_pe # (batch, seq, d_model)

        #pass src_emb and src_mask throught transformer encoder using self.transformer.encoder

        return self.transformer.encoder(src_emb, src_mask)


    def decode(self, tgt, memory,tgt_mask):
        """Method to decode a batch of data through the transformer model."""
        #pass tgt throught embedding layer
        scale = math.sqrt(self.d_model)
        tgt_emb = self.tgt_embedding(tgt) * scale #size -> (batch, seq_len, d_model)
        tgt_seq_len = tgt_emb.size(1)   # returns seq length

        #create positional encoding
        pe = self.positional_encoding(self.d_model, maxlen=tgt_seq_len).to(tgt.device) # (batch, seq, d_model)

        tgt_pe = pe[:, :tgt_seq_len, :] # [1, S_tgt, d_model]
        #add tgt_emb and tgt_pe
        #pass tgt_emb and tgt_mask throught transformer encoder using self.transformer.decoder
        tgt_emb = tgt_emb + tgt_pe # (batch, seq, d_model)
        memory = memory.to(tgt_emb.device)
        return self.transformer.decoder(tgt_emb, memory,tgt_mask)


Now that our model is ready, we still need some methods that will come in handy during training.

In [9]:
import torch
def create_padding_mask(seq):
  """ Method to create a padding mask based on given sequence.
  Arguments
  ---------
  seq : Tensor
     Sequence to create padding mask for
  Returns
  -------
  mask : Tensor
      Padding mask
  """
  return (seq == 0) #boolean matrix that is True when datapoint is equal to 0

def create_triu_mask(sz):
    """ Method to create a triangular mask based on given sequence. This is used for the tgt mask in the Transformer model to avoid looking ahead.
    Arguments
    ---------
    seq : Tensor
        Sequence to create triangular mask for
    Returns
    -------
    mask : Tensor
        Triangular mask

    Create a look-ahead mask (upper triangular) of shape [sz, sz].
    Arguments
    ---------
    sz : int
        Sequence length (target length).

    Returns
    -------
    mask : Tensor
        Float mask of shape [sz, sz], where:
          - masked positions = -inf
          - allowed positions = 0
    """

  # TODO
  #create triangular mask of size sz x sz
    mask = torch.triu(torch.ones(sz, sz), diagonal=1)
    # transpose (PyTorch docs recommend this for consistency)


  #tranpose mask and cast to float type
  #in pytorch the masked objects expect -inf instead of zero.
  #Replace all 0 for -inf and all 1's for 0's
    mask = mask.masked_fill(mask == 1, float('-inf'))
    mask = mask.masked_fill(mask == 0, 0.0)

    return mask

def tokenize_batch(source, targets,tokenizer):
  """ Method to tokenize a batch of data given a tokenizer.
  Arguments
  ---------
  source : List of String
     Source text
  targets : List of String
     Target text
  tokenizer : Tokenizer
     Tokenizer to use for tokenization
  Returns
  -------
  tokenized_source : Tensor
      Tokenized source text
  """

  tokenized_source = tokenizer(source, padding='max_length', max_length=128, return_tensors='pt', truncation=True)

  tokenized_targets = tokenizer(targets,  padding='max_length', max_length=128, return_tensors='pt',truncation=True)

  return tokenized_source['input_ids'], tokenized_targets['input_ids']


In [10]:
a=create_triu_mask(5)
b= torch.tensor([[0., float('-inf'), float('-inf'), float('-inf'), float('-inf')],
        [0., 0., float('-inf'), float('-inf'), float('-inf')],
        [0., 0., 0., float('-inf'), float('-inf')],
        [0., 0., 0., 0., float('-inf')],
        [0., 0., 0., 0., 0.]])
assert torch.equal(a,b), "Issues with create_triu_mask"
print("Good Job!")

Good Job!


### Training


In [11]:
from transformers import AutoTokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer=AutoTokenizer.from_pretrained('FacebookAI/xlm-roberta-base')
PAD_IDX = tokenizer.pad_token_id #for padding
BOS_IDX = tokenizer.bos_token_id #for beggining of sentence
EOS_IDX = tokenizer.eos_token_id #for end of sentence

model = TransformerModel(tokenizer.vocab_size, tokenizer.vocab_size,512, 8, 3, 3, 256,0.1).to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
loss_function = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

train_loader = torch.utils.data.DataLoader(training_set, batch_size=8, shuffle=True) #change batch size based on your reasources
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=8, shuffle=False) #change batch size based on your reasources

In [12]:
from transformers import AutoTokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer=AutoTokenizer.from_pretrained('FacebookAI/xlm-roberta-base')

In [13]:
from torch.utils.data import DataLoader
from tqdm import tqdm

def train_epoch(model,train_loader,tokenizer):
    model.train()
    losses = 0

    for src, tgt in tqdm(train_loader):
        src, tgt = tokenize_batch(src, tgt, tokenizer)
        src = src.to(device) # tensor of shape [batch_size, seq_len]
        tgt = tgt.to(device) # tensor of shape [batch_size, seq_len]

        tgt_input = tgt[:,:-1]

        src_seq_len = src.size(1)

        #creat src_mask this is basically a matrix of 0s of shape Sequence x Sequence
        src_mask = torch.zeros((src_seq_len, src_seq_len), device=device)
        tgt_seq_len = tgt_input.size(1)
        tgt_mask = create_triu_mask(tgt_seq_len).to(device) #create triangular mask for target

        src_padding_mask = create_padding_mask(src).to(device) #create padding mask for src
        tgt_padding_mask = create_padding_mask(tgt_input).to(device) #create padding mask for tgt

        logits = model(
    src,
    tgt_input,
    src_mask,
    tgt_mask,
    src_padding_mask,
    tgt_padding_mask,
)#pass it through model

        optimizer.zero_grad()

        tgt_out = tgt[:,1:]
        loss = loss_function(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_loader))


def evaluate(model,val_dataloader):
    model.eval()
    losses = 0
    with torch.no_grad():
      for src, tgt in tqdm(val_dataloader):
          src, tgt = tokenize_batch(src, tgt, tokenizer)
          src = src.to(device)
          tgt = tgt.to(device)

          tgt_input = tgt[:,:-1]

          #do the same as in Train
          src_seq_len = src.size(1)

          #creat src_mask this is basically a matrix of 0s of shape Sequence x Sequence
          src_mask = torch.zeros((src_seq_len, src_seq_len), device=device)
          tgt_seq_len = tgt_input.size(1)
          tgt_mask = create_triu_mask(tgt_seq_len).to(device) #create triangular mask for target

          src_padding_mask = create_padding_mask(src).to(device) #create padding mask for src
          tgt_padding_mask = create_padding_mask(tgt_input).to(device) #create padding mask for tgt

          logits = model(
      src,
      tgt_input,
      src_mask,
      tgt_mask,
      src_padding_mask,
      tgt_padding_mask,
  )#pass it through model

          tgt_out = tgt[:,1:]
          loss = loss_function(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
          losses += loss.item()

    return losses / len(list(val_dataloader))

Now we can start training! Keep in mind this code is very demanding computationally, it has been set to 10 epochs (which can take up to 4 hours) but feel free to change this value depending on your resources, in this case the more epochs you can execute the better 😀

In [14]:
def train(model, epochs, train_loader,validation_loader ):
  for epoch in range(1, epochs+1):
        train_loss = train_epoch(model,train_loader, tokenizer)
        val_loss = evaluate(model,validation_loader)
        print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}"))

#train(model, 5, train_loader,validation_loader)

In [15]:
from google.colab import drive
#drive.mount('/content/drive')
#path = f"/content/drive/MyDrive/ENG_SPA_Transformer_Translator_Model.pth"
#torch.save(model.state_dict(), path)
state_dict = torch.load("/content/drive/MyDrive/ENG_SPA_Transformer_Translator_Model.pth", map_location=device)
model.load_state_dict(state_dict)
model.to(device)

TransformerModel(
  (src_embedding): Embedding(250002, 512)
  (tgt_embedding): Embedding(250002, 512)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-2): 3 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_features=512, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=256, out_features=512, bias=True)
          (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(


### Testing


We will use three different evaluation metrics to see our model's test performance: [Bert Score](https://huggingface.co/spaces/evaluate-metric/bertscore), [Meteor](https://huggingface.co/spaces/evaluate-metric/meteor) and [Rouge](https://huggingface.co/spaces/evaluate-metric/rouge). Please access their hugging face documentation to know how to implement them.

In [16]:
from evaluate import load

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


Implement greedy decode seen in class

In [17]:

# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)
    memory = model.encode(src, src_mask) #pass src through encoder
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):

        tgt_mask = create_triu_mask(ys.size(1)).to(device) #create triangular mask
        out = model.decode(tgt=ys, memory=memory, tgt_mask=tgt_mask)#pass through decoder

        prob = model.fc(out[:, -1, :])

        #get next word based on probabilities (remember to use .item())
        next_word = torch.argmax(prob, dim=-1).item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
        if next_word == EOS_IDX:
            break
    return ys

def translate(model: torch.nn.Module, src_sentence: str, tokenizer):
    model.eval()
    src, _ = tokenize_batch(src_sentence, "", tokenizer)
    src = src.to(device)
    num_tokens = src.shape[1] # sentence len
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.float).to(device)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len= int(num_tokens * 1.2 ), start_symbol=BOS_IDX).flatten()
    return tokenizer.decode(tgt_tokens, skip_special_tokens=True)

In [18]:
print(translate(model, "hello how are you today",tokenizer))

¿por qué te habéis hecho?


In [19]:
test_set = test_set[:1000]

In [22]:
import numpy as np
# you can also trim test_loader
bertscore = load("bertscore")
rouge = load('rouge')
meteor = load('meteor')

def test(test_loader, model, tokenizer, device, max_length=200):
  """Method to test our model using best score and meteor metric.
  Arguments
  ---------
  test_loader: Dataloader
    Dataloader that holds test set
  model: nn.Module
    trained Machine Translation model
  tokenizer:
  """
  model.to(device)
  model.eval()
  precision = 0.0
  recall = 0.0
  f1 = 0.0
  meteor_metric = 0.0
  n_samples = 0
  with torch.no_grad():
    for src, target in test_loader:
      #Use translade method to evaluate our model
      pred = translate(model, src, tokenizer)  # prediction string

      #get results bert
      results_bert = bertscore.compute(
          predictions=[pred],
          references=[target],
          lang="en"
      )

      results_meteor = meteor.compute(
      predictions=[pred],
      references=[target]
      ) #get results meteor
      precision += results_bert["precision"][0] #get precision of results_bert
      recall += results_bert["recall"][0] #get recall of results_bert
      f1 += results_bert["f1"][0] #get f1 of results_bert
      meteor_metric+= results_meteor["meteor"] #get meteor metric of results_meteor
      n_samples += 1
  return precision / len(test_loader), recall / len(test_loader), f1 / len(test_loader), meteor_metric / len(test_loader)

test(test_set, model, tokenizer, device)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(0.8171997967362404,
 0.8065623074173928,
 0.8112730476856231,
 np.float64(0.1402645833974278))

In [24]:
print("\n=== Translation Evaluation Metrics ===")
print(f"BERTScore Precision: {0.81719:.2f}")
print(f"BERTScore Recall:    {0.80656230:.2f}")
print(f"BERTScore F1:        {0.8112730476:.2f}")
print(f"METEOR:              {0.140264583397:.2f}")


=== Translation Evaluation Metrics ===
BERTScore Precision: 0.8172
BERTScore Recall:    0.8066
BERTScore F1:        0.8113
METEOR:              0.1403


## Let's experiment!

Pick 2 experiments out these 3:
1. Play with a hyperparameter of your choice to measure its effect on the translation.

2. Train an inverse translator from spanish to english and compare the performance.

3. Compare the results of your model with the performance of using the T5 pretrained model. This [tutorial](https://huggingface.co/docs/transformers/en/tasks/translation) on using T5 for machine translation might come in handy.



